In [2]:
!pip uninstall -y numpy networkx unsloth unsloth_zoo transformers
!pip install "numpy>=1.26.0,<2.0"
!pip install --upgrade networkx
!pip install unsloth

Found existing installation: numpy 1.24.4
Uninstalling numpy-1.24.4:
  Successfully uninstalled numpy-1.24.4
Found existing installation: networkx 2.4
Uninstalling networkx-2.4:
ERROR: Exception:
Traceback (most recent call last):
  File "/usr/lib/python3.10/shutil.py", line 816, in move
    os.rename(src, real_dst)
PermissionError: [Errno 13] Permission denied: '/usr/lib/python3/dist-packages/networkx' -> '/tmp/pip-uninstall-ouw_ql2l'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.10/site-packages/pip/_internal/cli/base_command.py", line 107, in _run_wrapper
    status = _inner_run()
  File "/home/ubuntu/.local/lib/python3.10/site-packages/pip/_internal/cli/base_command.py", line 98, in _inner_run
    return self.run(options, args)
  File "/home/ubuntu/.local/lib/python3.10/site-packages/pip/_internal/commands/uninstall.py", line 105, in run
    uninstall_pathset = req.uninstall(
  File "

In [1]:
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
print("Loading model...")
max_seq_length = 2048
dtype = None  
load_in_4bit = False

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    device_map="auto"
)
print("✓ Model loaded")

Loading model...
==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 5.0.0.dev0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.495 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.44it/s]


✓ Model loaded


In [3]:
print("\nConfiguring LoRA adapters...")
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 42,
)
print("✓ LoRA configured")


Configuring LoRA adapters...


Unsloth 2025.11.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


✓ LoRA configured


In [4]:
print("\nLoading dataset...")
dataset = load_dataset("json", data_files="coqa_mctest_5000_samples.jsonl", split="train")
print(f"✓ Loaded {len(dataset)} samples")

train_test_split = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]
print(f"✓ Dataset loaded: {len(dataset)} examples")


Loading dataset...
✓ Loaded 5000 samples
✓ Dataset loaded: 5000 examples


In [5]:
print("\nSetting up trainer...")
TOTAL_SAMPLES = len(train_dataset)
GLOBAL_BATCH_SIZE = 2 * 4
TRAIN_STEPS = TOTAL_SAMPLES // GLOBAL_BATCH_SIZE
print(f"Training configuration:")
print(f"  - Total samples: {TOTAL_SAMPLES}")
print(f"  - Global batch size: {GLOBAL_BATCH_SIZE}")
print(f"  - Training steps: {TRAIN_STEPS}")
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=SFTConfig(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=TRAIN_STEPS,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        output_dir="outputs",
        eval_strategy="steps",
        eval_steps=100,
        save_strategy="steps",
        save_steps=100,
        per_device_eval_batch_size=4,
        load_best_model_at_end=True,
        save_total_limit=2,
        dataloader_num_workers=0
    ),
)
print("✓ Trainer configured")

warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.



Setting up trainer...
Training configuration:
  - Total samples: 4500
  - Global batch size: 8
  - Training steps: 562
✓ Trainer configured


In [6]:
print("\nStarting model training...")
trainer.train()
print("✓ Training finished!")

print("\nMerging LoRA adapters into base model...")
model = model.merge_and_unload()
print("✓ LoRA adapters merged")

The model is already on multiple devices. Skipping the move to device specified in `args`.



Starting model training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,500 | Num Epochs = 2 | Total steps = 562
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
100,1.297100,1.258698
200,0.365900,0.369434
300,0.123000,0.130474
400,0.078300,0.087987
500,0.086600,0.076535


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


✓ Training finished!

Merging LoRA adapters into base model...
✓ LoRA adapters merged


In [7]:
model.save_pretrained("llama-3.1-8b-finetuned")
tokenizer.save_pretrained("llama-3.1-8b-finetuned")

('llama-3.1-8b-finetuned/tokenizer_config.json',
 'llama-3.1-8b-finetuned/special_tokens_map.json',
 'llama-3.1-8b-finetuned/chat_template.jinja',
 'llama-3.1-8b-finetuned/tokenizer.json')

In [8]:
model.save_pretrained_gguf(
    "llama-3.1-8b-finetuned",
    tokenizer,
    quantization_method="q4_k_m",
)

Unsloth: Merging model weights to 16-bit format...


/home/ubuntu/.local/lib/python3.10/site-packages/unsloth_zoo/saving_utils.py:969: UserWarning: Model is not a PeftModel (no Lora adapters detected). Skipping Merge. Please use save_pretrained() or push_to_hub() instead!
  warnings.warn("Model is not a PeftModel (no Lora adapters detected). Skipping Merge. Please use save_pretrained() or push_to_hub() instead!")


Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF bf16 might take 3 minutes.
\        /    [2] Converting GGUF bf16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: llama.cpp found in the system. Skipping installation.
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into bf16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['Meta-Llama-3.1-8B-Instruct.BF16.gguf']
Unsloth: [2] Converting GGUF bf16 into q4_k_m. This might take 10 minutes...
Unsloth: Model files cleanup...
Unsloth: All GGUF conversions completed successfully!
Generated files: ['Meta-Llama-3.1-8B-Instruct.Q4_K_M.gguf']
Unsloth: example usage for text only LLMs: llama-cli --model Meta-Llama-3.1-8B-Instruct.Q4_K_M.gguf -p "why is the sky blue?"
Uns

{'save_directory': 'llama-3.1-8b-finetuned',
 'gguf_files': ['Meta-Llama-3.1-8B-Instruct.Q4_K_M.gguf'],
 'modelfile_location': '/home/ubuntu/Modelfile',
 'want_full_precision': False,
 'is_vlm': False,
 'fix_bos_token': False}